In [70]:
# Import necessary functions

import requests
import pandas as pd
import numpy as np
import itertools

In [2]:
def obtain_traits(token_no, nft_contract):
  """

  This is a function to obtain traits and their values from OpenSea
  Inputs:
  token_no - int, token id of an NFT collection
  nft_contract - str, nft contract
  -------------------
  Output:
  zipped - list, list of trait types and their values zipped together
  no_of_traits - int, the number of traits of a certain token

  """
  single_asset_url = "https://api.opensea.io/asset/" + nft_contract + "/" + str(token_no) # Create OpenSea API URL
  single_asset = requests.get(single_asset_url).json()
  token_traits = single_asset['traits'] # Store trait data from OpenSea API
  no_of_traits = len(token_traits) # Count the number of traits
  trait_types = []
  trait_values = []
  # Retrieve trait types
  for i in range(no_of_traits): 
      trait_types.append(token_traits[i]['trait_type']) #will be the column
      trait_values.append(token_traits[i]['value']) #will be the column values
  zipped = list(zip(trait_types, trait_values))
  return zipped, no_of_traits

In [3]:
def outer_merge_prep(zippedlist, token_number, number_of_traits):

  """
  This is a function to change zipped list format for each token in an NFT collection
  to dataframe format
  Inputs:
  zippedlist - zipped list, a zipped list containing trait types and their values
  token_number - int, the number of id of a token
  number_of_traits - int, the number of traits of a certain token
  --------------------
  Output:
  df_prep - dataframe, a dataframe containing trait list and their values of a certain
  token
  """

  df_prep = pd.DataFrame(zippedlist)
  df_prep = df_prep.T
  df_prep.columns = df_prep.iloc[0]
  df_prep = df_prep[1:]
  df_prep['Token_id'] = token_number
  df_prep['Trait_count'] = number_of_traits


  return df_prep

In [77]:
# Input collection data 

contract = "0xED5AF388653567Af2F388E6224dC7C4b3241C544" #Azuki contract
collection_size = 10000 #Azuki collection size

In [79]:
n = 100
max_part = int(collection_size/n)

In [80]:
placeholder = list(itertools.repeat([0],n))

In [81]:
for j in range(0,n):
    token_id =  int(0 + max_part*j)
    zipped, no_of_traits = obtain_traits(token_id, contract)
    df1 = outer_merge_prep(zipped, token_id, no_of_traits)
    outer_merge = df1

    for i in range(1, max_part):
      token_id = int(i + j * max_part)
      zipped, no_of_traits = obtain_traits(token_id, contract)
      df2 = outer_merge_prep(zipped, token_id, no_of_traits)
      outer_merge = outer_merge.merge(df2, how = 'outer') #merge from 0 to 9
    placeholder[j] = outer_merge

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [82]:
final_dataset = placeholder[0]
for k in range(1,n):
  final_dataset = final_dataset.merge(placeholder[k], how = 'outer') 

In [83]:
final_dataset

,Hair,Clothing,Background,Mouth,Offhand,Type,Eyes,Token_id,Trait_count,Ear,Neck,Headgear,Face,Special
0,Water,Pink Oversized Kimono,Off White A,Frown,Monkey King Staff,Human,Striking,0,7,NaN,NaN,NaN,NaN,NaN
1,Pink Hairband,White Qipao with Fur,Off White D,Lipstick,Gloves,Human,Daydreaming,1,7,NaN,NaN,NaN,NaN,NaN
2,Pink Flowy,Vest,Red,Chewing,NaN,Human,Ruby,2,7,Red Tassel,NaN,NaN,NaN,NaN
3,Green Spiky,Green Yukata,Red,Grass,Katana,Human,Careless,3,9,NaN,Frog Headphones,Frog Headband,NaN,NaN
4,Brown Dreadlocks,White Qipao with Fur,Off White D,Smirk,Katana,Human,Lightning,4,7,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Magenta Long,Black Kimono,Off White A,Tactical Knife,NaN,Human,Suspicious,9995,7,NaN,NaN,Full Bandana,NaN,NaN
9996,Blonde Swept Back,Black Kimono,Off White A,Meh,Skateboard,Human,Pensive,9996,9,NaN,Chill Headphones,NaN,Reading Glasses,NaN
9997,Brown Messy,Azuki Track Jacket,Off White B,Not Bad,Sakura Katana,Human,Curious,9997,7,NaN,NaN,NaN,NaN,NaN
9998,Orange Swept Back,Suit with Turtleneck,Off White A,Chuckle,Fireball,Human,Closed,9998,9,NaN,NaN,NaN,Eye Patch,Fire


In [85]:
final_dataset.to_csv('azuki_traits_dataframe.csv')

In [93]:
gender = pd.read_csv (r'https://raw.githubusercontent.com/fcitra/Azuki_traits/main/azuki_gender_dataset.csv')
gender.head()

,Unnamed: 0,Token_id,Trait_count,Trait_list,Three categories,Two categories
0,0,0,7,"Water, Pink Oversized Kimono, Off White A, Fro...",F,F
1,1,1,7,"Off White D, Daydreaming, Pink Hairband, White...",F,F
2,2,2,7,"Red, Pink Flowy, Chewing, Ruby, Vest, Human, R...",F,F
3,3,3,9,"Red, Green Spiky, Frog Headphones, Katana, Gra...",M,M
4,4,4,7,"Smirk, Brown Dreadlocks, Off White D, Katana, ...",A,F


In [94]:
gender.columns

Index(['Unnamed: 0', 'Token_id', 'Trait_count', 'Trait_list',
       'Three categories', 'Two categories'],
      dtype='object')

In [95]:
gender = gender.drop(['Unnamed: 0','Trait_count', 'Trait_list', 'Three categories'], axis=1)

In [96]:
traits_with_gender = gender.merge(final_dataset, how = 'outer') 

In [97]:
traits_with_gender.head()

,Token_id,Two categories,Hair,Clothing,Background,Mouth,Offhand,Type,Eyes,Trait_count,Ear,Neck,Headgear,Face,Special
0,0,F,Water,Pink Oversized Kimono,Off White A,Frown,Monkey King Staff,Human,Striking,7,NaN,NaN,NaN,NaN,NaN
1,1,F,Pink Hairband,White Qipao with Fur,Off White D,Lipstick,Gloves,Human,Daydreaming,7,NaN,NaN,NaN,NaN,NaN
2,2,F,Pink Flowy,Vest,Red,Chewing,NaN,Human,Ruby,7,Red Tassel,NaN,NaN,NaN,NaN
3,3,M,Green Spiky,Green Yukata,Red,Grass,Katana,Human,Careless,9,NaN,Frog Headphones,Frog Headband,NaN,NaN
4,4,F,Brown Dreadlocks,White Qipao with Fur,Off White D,Smirk,Katana,Human,Lightning,7,NaN,NaN,NaN,NaN,NaN


In [98]:
traits_with_gender.to_csv('azuki_traits_with_gender_df.csv')